# µNotebook Stress Test

This notebook contains **MicroPython-style** code snippets designed to stress-test a notebook frontend: large outputs, unicode, ANSI color, errors, long lines, binary-ish data, and incremental logging.

- You can run cells individually to verify streaming, truncation, and rendering.
- The code is written to be compatible with standard Python syntax that MicroPython also supports (adjust imports like `utime` vs `time` as needed by your runtime).


In [ ]:
print('Hello World!')


In [ ]:
# 1) Hello + quick sanity check (UTF-8, symbols)
print('Hello from MicroPython stress test!')
print('Greek mu: µ  |  Emoji: 🚀  |  Accents: café  |  RTL: مرحبا')


In [ ]:
# 2) Rapid 1000-line output (checks line buffering & scroll performance)
for i in range(1000):
    print('line', i)


In [ ]:
# 3) Progress stream with carriage returns (\r) to test in-place updates
try:
    import utime as time
    sleep = time.sleep
except ImportError:
    try:
      import time
      sleep = time.sleep
    except ImportError:
      from pybricks.tools import wait
      sleep = wait

for i in range(0, 101):
    bar = '#' * (i // 2)
    s = '\rProgress: {0:3d}% |{1:<50}|'.format(i, bar)
    print(s, end='')
    sleep(0.1)
print('\nDone.')


In [ ]:
# 4) Very long line (10k+ characters) to test wrapping/truncation
long_line = 'A' * 10_000
print(long_line)
print('Length:', len(long_line))


In [ ]:
# 5) ANSI color / style sequences (if your UI supports them)
RESET='\x1b[0m'
BOLD='\x1b[1m'
RED='\x1b[31m'
GREEN='\x1b[32m'
YELLOW='\x1b[33m'
BLUE='\x1b[34m'
print(BOLD + 'Bold line' + RESET)
print(RED + 'Red' + RESET, GREEN + 'Green' + RESET, YELLOW + 'Yellow' + RESET, BLUE + 'Blue' + RESET)
for i in range(6):
    print('\x1b[3%dmColor %d%s' % (i, i, RESET))


In [ ]:
# 6) Unicode torture: combining marks, wide glyphs, RTL, emoji sequences
samples = [
    'Combining: A\u0301 A\u0323 A\u0301\u0323',
    'CJK: 你好，世界',
    'RTL (Arabic): العربية مرحبا',
    'Emoji ZWJ: 👩\u200d💻 👨\u200d🚀',
    'Skin tones: 👍👍🏻👍🏼👍🏽👍🏾👍🏿',
    'Mu: µ — microfarad µF'
]
for s in samples:
    print(s)


In [ ]:
# 7) Binary-like output (NULs and non-printables) using repr to stay text-safe
import usys as sys
buf = bytes([i % 256 for i in range(1024)])
sys.stdout.write('bytes len: %i\n' % len(buf))
sys.stdout.write(repr(buf))
sys.stdout.write('\n')

# Also write some control codes directly to test your renderer
ctrls = b'CTRL-B:\x02 CTRL-C:\x03 NUL:\x00 END\n' 
sys.stdout.write(ctrls)


In [ ]:
# 8) Intentional exceptions to test traceback rendering and stderr capture
def boom():
    raise ValueError('Intentional error for traceback test')

try:
    boom()
except Exception as exc:
    print('Caught:', exc)
    # Now raise unhandled to see a raw traceback
    raise exc


In [ ]:
# 9) Large JSON-ish output to test pretty-print and folding
try:
    import ujson as json
except ImportError:
    import json

data = {
    'items': [ {'i': i, 'txt': 'x'*200 } for i in range(50) ],
    'meta': {'created': 'now', 'u': 'mu', 'note': 'stress-test'}
}
print(json.dumps(data)[:5000])  # print only first 5k chars


In [ ]:
# 10) Streaming logs with flush to test incremental delivery in your UI
try:
    import utime as time
    sleep = time.sleep
except ImportError:
    try:
      import time
      sleep = time.sleep
    except ImportError:
      from pybricks.tools import wait
      sleep = wait

for i in range(10):
    print('tick', i)
    try:
        import sys
        sys.stdout.flush()
    except Exception:
        pass
    sleep(.5)


In [ ]:
# 11) Explicit stderr writes if your kernel/bridge routes them distinctly
try:
    import usys as sys
    sys.stderr.write('This is stderr line 1\n')
    sys.stderr.write('This is stderr line 2\n')
except Exception as e:
    print('stderr not available:', e)


In [ ]:
# 12) Longer runtime with intermittent output (cancel-friendly)
for i in range(10):
    print('chunk', i, '...' )
    for j in range(50000):
        pass  # burn CPU briefly
print('long runtime section finished')

In [ ]:
# 13) A long program
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')
print('hello world!!!!!!!!!!')


In [ ]:
# very wide code in editor

try: import json
except: import ujson as json
def unotebook_repr(o):
  if hasattr(o, '_repr_mimebundle_'): print(json.dumps(o._repr_mimebundle_()), end='')
  else:
    print('{', end='')
    if hasattr(o, '_repr_html_'):       print('"text/html":', json.dumps(o._repr_html_()), end=',')
    if hasattr(o, '_repr_markdown_'):   print('"text/markdown":', json.dumps(o._repr_html_()), end=',')
    if hasattr(o, '_repr_svg_'):        print('"image/svg+xml":', json.dumps(o._repr_svg_()), end=',')
    if hasattr(o, '_repr_png_'):        print('"image/png":', json.dumps(o._repr_png_()), end=',')
    if hasattr(o, '_repr_jpeg_'):       print('"image/jpeg":', json.dumps(o._repr_jpeg_()), end=',')
    if hasattr(o, '_repr_latex_'):      print('"text/latex":', json.dumps(o._repr_latex_()), end=',')
    if hasattr(o, '_repr_javascript_'): print('"application/javascript":', json.dumps(o._repr_javascript_()), end=',')
    print(json.dumps({'text/plain':repr(o)}), end='}\n')
    

In [ ]:
class Bold:
  def _repr_html_(self):
    return '<b>Bold</b>'

Bold()

In [ ]:
# 16x16 black square
jpeg_bytes = b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xdb\x00C\x01\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xc0\x00\x11\x08\x00\x10\x00\x10\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc2\xc3\xc4\xc5\xc6\xc7\xc8\xc9\xca\xd2\xd3\xd4\xd5\xd6\xd7\xd8\xd9\xda\xe1\xe2\xe3\xe4\xe5\xe6\xe7\xe8\xe9\xea\xf1\xf2\xf3\xf4\xf5\xf6\xf7\xf8\xf9\xfa\xff\xc4\x00\x1f\x01\x00\x03\x01\x01\x01\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x11\x00\x02\x01\x02\x04\x04\x03\x04\x07\x05\x04\x04\x00\x01\x02w\x00\x01\x02\x03\x11\x04\x05!1\x06\x12AQ\x07aq\x13"2\x81\x08\x14B\x91\xa1\xb1\xc1\t#3R\xf0\x15br\xd1\n\x16$4\xe1%\xf1\x17\x18\x19\x1a&\'()*56789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x82\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc2\xc3\xc4\xc5\xc6\xc7\xc8\xc9\xca\xd2\xd3\xd4\xd5\xd6\xd7\xd8\xd9\xda\xe2\xe3\xe4\xe5\xe6\xe7\xe8\xe9\xea\xf2\xf3\xf4\xf5\xf6\xf7\xf8\xf9\xfa\xff\xda\x00\x0c\x03\x01\x00\x02\x11\x03\x11\x00?\x00\x8e\x8a(\xa0\x0f\xff\xd9'

class JPEG:
  def _repr_jpeg_(self):
    return jpeg_bytes

JPEG()

In [ ]:
# 16x16 black square
png_bytes = b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x10\x00\x00\x00\x10\x08\x02\x00\x00\x00\x90\x91h6\x00\x00\x00\x10IDATx\x9cc`\x18\x05\xa3`\x14\xc0\x00\x00\x03\x10\x00\x01?,+\xec\x00\x00\x00\x00IEND\xaeB`\x82'

class PNG:
  def _repr_png_(self):
    return png_bytes

PNG()